## LLMをプログラムで操作する

これまでにChatGPTのような大規模言語モデル（LLM）とインターフェースしたことがあると思います。通常は、UIやアプリケーションを通じて行います。

このノートブックでは、Pythonを使用してLLMに接続し、APIを介して直接クエリを行います。このラボでは、モデルとして**Mistral-7B Instruct v2**（https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2）を選択しました。このモデルは完全にオープンソース（Apache 2.0ライセンス）で、他の商用やオープンソースモデルに比べて軽量ですが、我々が使用しようとしているタスクにおいて非常に高性能です。

このモデルは既にラボクラスターにデプロイされています。小型モデルとはいえ、動作には24GBのRAMを持つGPUが必要です。

### 必要条件とインポート

ラボの指示に従って正しいワークベンチイメージを選択した場合は、必要なライブラリがすでにインストールされています。もしそうでない場合は、次のセルの最初の行のコメントを外して、必要なパッケージをインストールしてください。続いて必要なライブラリをインポートします。

In [ ]:
# !pip install --no-cache-dir --no-dependencies --disable-pip-version-check -r requirements.txt # 正しいワークベンチイメージを選択していない場合のみ、コメントを外してください

import json
import os
from os import listdir
from os.path import isfile, join
from langchain.chains import LLMChain
from langchain_community.llms import VLLMOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate

### Langchain

Langchain (https://www.langchain.com/) は、言語モデルを利用したアプリケーション開発のためのフレームワークです。これにより、LLMを適切にクエリするための面倒なボイラープレートコードを書く必要がなくなります。

まず、LLMインスタンスを作成します。これはLLM APIがクエリできる場所と、モデルに適用されるいくつかのパラメータによって定義されます。例えば、`max_new_tokens`はモデルが最大512トークン（単語または単語の一部）まで回答するように指示します。`temperature`は非常に低く設定されており、モデルがあまり「クリエイティブ」にならないようにし、事実に基づいた回答を行うようにします。結局のところ、ここでは派手な詩を書こうとしているわけではありません！

In [ ]:
# LLM推論サーバーのURL
inference_server_url = "http://llm.ic-shared-llm.svc.cluster.local:8000"

# LLMの定義
llm = VLLMOpenAI(           # vLLMのOpenAI互換APIクライアントを使用しています。しかし、モデルはOpenShift上で実行されています。
    openai_api_key="EMPTY",   # したがって、OpenAIキーは必要ありません。
    openai_api_base= f"{inference_server_url}/v1",
    model_name="mistralai/Mistral-7B-Instruct-v0.2",
    top_p=0.92,
    temperature=0.01,
    max_tokens=512,
    presence_penalty=1.03,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

また、送信するリクエストごとに適用される**テンプレート**（「プロンプト」）も必要です。

モデルにクエリを送る際、ユーザーが入力した内容をそのまま送ることはほとんどありません。この入力に加えて、モデルがどのようにそれを処理すべきか、回答方法や回答しない内容、使用する口調などの適切な指示をモデルに与える必要があります。

In [ ]:
template="""<s>[INST]<<SYS>>
あなたは役立つ、尊敬を持った、誠実なアシスタントです。可能な限り有益であり、かつ安全であることが求められます。
質問が行われますので、それに答えてください。
回答には有害、非倫理的、人種差別的、性差別的、有毒、危険、または違法な内容を含めないでください。
あなたの回答は社会的に偏見のないもので、前向きであるべきです。
質問が意味不明である場合や、事実として一貫性がない場合は、誤った回答をするのではなく、その理由を説明してください。
質問の答えがわからない場合は「わかりません」と答えてください。
<</SYS>>

### 質問:
{input}

### 回答:
[/INST]
"""
PROMPT = PromptTemplate(input_variables=["input"], template=template)

Langchainを使用すると、これらの要素を簡単に「結びつけ」て**会話**オブジェクトを作成し、モデルにクエリを送信することができます。

In [ ]:
conversation = LLMChain(llm=llm,
                        prompt=PROMPT,
                        verbose=False
                        )

これでモデルにクエリを送信する準備が整いました！

In [ ]:
query = "人工知能とは何ですか？"

conversation.predict(input=query); # 行の最後に";"を追加することで、最終出力（ストリームされた回答の繰り返し）を隠すことができます

このノートブックのセクション3.7に戻ると、いくつかのオプション演習がありますので、試してみてください。